## Imports

In [2]:
import pandas as pd
import numpy as np
import catboost
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

/home/anand173/.conda/envs/cent7/2024.02-py311/llama2/lib/python3.11/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [3]:
bracket_training = pd.read_csv('/home/anand173/code/bracket_training_with_names.csv')
bracket_training.head()

,Unnamed: 0,CustomerID,CustomerAreaCode,CustomerPostalCode,CustomerPostalCodeLatitude,CustomerPostalCodeLongitude,CustomerDMACode,CustomerDMADescription,NCAACustomerRecordCreated,BracketEntryId,BracketEntryCreatedDate,RegionWinner_East,RegionWinner_West,RegionWinner_South,RegionWinner_Midwest,SemifinalWinner_East_West,SemifinalWinner_South_Midwest,NationalChampion
0,0,47028,NaN,36093,32.5622,-86.0994,698.0,MONTGOMERY (SELMA),2021-12-25,1723503,2024-03-19 10:27:15 -0400,UConn,Arizona,James Madison,Tennessee,UConn,Tennessee,Tennessee
1,1,3511,616.0,49464,42.8256,-86.0104,563.0,GRAND RAPIDS - KALMZOO - B. CRK,2021-04-02,963479,2024-03-18 10:16:39 -0400,UConn,Baylor,Kentucky,Kansas,UConn,Kentucky,UConn
2,2,58445,703.0,22210,38.8808,-77.1129,511.0,"WASHINGTON, DC (HAGRSTWN)",2021-04-02,810038,2024-03-18 00:21:47 -0400,UConn,Baylor,Houston,Purdue,UConn,Purdue,Purdue
3,3,28833,NaN,78218,29.4969,-98.4032,641.0,SAN ANTONIO,2023-11-16,3384825,2024-03-21 10:28:56 -0400,Iowa St.,Saint Mary's,NC State,Purdue,Iowa St.,NC State,NC State
4,4,37899,NaN,14212,42.8946,-78.8245,514.0,BUFFALO,2022-03-16,2828017,2024-03-20 20:14:52 -0400,Auburn,North Carolina,Marquette,Creighton,North Carolina,Marquette,North Carolina


In [4]:
bracket_test = pd.read_csv('/home/anand173/code/bracket_test_with_names.csv')
bracket_test['SemifinalWinner_East_West'] = ""
bracket_test['SemifinalWinner_South_Midwest'] = ""
bracket_test.head()

,Unnamed: 0,CustomerID,CustomerAreaCode,CustomerPostalCode,CustomerPostalCodeLatitude,CustomerPostalCodeLongitude,CustomerDMACode,CustomerDMADescription,NCAACustomerRecordCreated,BracketEntryId,BracketEntryCreatedDate,RegionWinner_East,RegionWinner_West,RegionWinner_South,RegionWinner_Midwest,SemifinalWinner_East_West,SemifinalWinner_South_Midwest
0,0,73662,919.0,27539,35.7225,-78.8408,560.0,RALEIGH - DURHAM (FAYETVLLE),3/29/20,2074118,3/19/24 18:50,UConn,North Carolina,Houston,Purdue,,
1,1,6679,360.0,97206,45.4840,-122.5973,820.0,"PORTLAND, OR",4/2/24,2692634,3/20/24 16:56,UConn,North Carolina,Duke,Kansas,,
2,2,63024,270.0,42754,37.4603,-86.3249,529.0,LOUISVILLE,12/8/21,1252684,3/18/24 15:13,Iowa St.,Arizona,Kentucky,Creighton,,
3,3,60371,206.0,98178,47.4924,-122.2359,819.0,SEATTLE - TACOMA,3/22/23,1950205,3/19/24 15:21,UConn,North Carolina,Houston,Purdue,,
4,4,18415,717.0,19038,40.1096,-75.1550,504.0,PHILADELPHIA,2/20/24,2756293,3/20/24 18:40,UConn,North Carolina,Marquette,Creighton,,


In [ ]:
bracket_training.shape

(130002, 18)

In [ ]:
bracket_test.shape

(14445, 17)

## Semifinal 1 Predictions

In [9]:
bracket_training_simple = bracket_training[['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_West', 'SemifinalWinner_East_West', 'BracketEntryId', 'NCAACustomerRecordCreated', 'BracketEntryCreatedDate']]
bracket_training_simple['CustomerDMACode'] = bracket_training_simple['CustomerDMACode'].astype(str).str[:-2]

# Ensure datetime columns are converted to datetime type, handling errors
bracket_training_simple['NCAACustomerRecordCreated'] = pd.to_datetime(bracket_training_simple['NCAACustomerRecordCreated'], errors='coerce')
bracket_training_simple['BracketEntryCreatedDate'] = pd.to_datetime(bracket_training_simple['BracketEntryCreatedDate'], errors='coerce')

# Calculate the number of months difference, handling NaT values
bracket_training_simple['MonthsSinceRecordCreated'] = (
    (bracket_training_simple['BracketEntryCreatedDate'].dt.year.fillna(0).astype(int) - bracket_training_simple['NCAACustomerRecordCreated'].dt.year.fillna(0).astype(int)) * 12 +
    (bracket_training_simple['BracketEntryCreatedDate'].dt.month.fillna(0).astype(int) - bracket_training_simple['NCAACustomerRecordCreated'].dt.month.fillna(0).astype(int))
)

# Drop the original datetime columns
bracket_training_simple.drop(['NCAACustomerRecordCreated', 'BracketEntryCreatedDate'], axis=1, inplace=True)

bracket_training_simple.set_index('BracketEntryId', inplace=True)
bracket_training_simple.head()

/tmp/ipykernel_90503/3663082735.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bracket_training_simple['CustomerDMACode'] = bracket_training_simple['CustomerDMACode'].astype(str).str[:-2]
/tmp/ipykernel_90503/3663082735.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bracket_training_simple['NCAACustomerRecordCreated'] = pd.to_datetime(bracket_training_simple['NCAACustomerRecordCreated'], errors='coerce')
/tmp/ipykernel_90503/3663082735.py:6: FutureWarning: In a future version of pandas, parsing da

AttributeError: Can only use .dt accessor with datetimelike values

In [8]:
bracket_test_simple = bracket_test[['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_West', 'SemifinalWinner_East_West', 'BracketEntryId']]
bracket_test_simple['CustomerDMACode'] = bracket_test_simple['CustomerDMACode'].astype(str).str[:-2]
bracket_test_simple.set_index('BracketEntryId', inplace=True)
bracket_test_simple.head()

/tmp/ipykernel_12998/2724997793.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bracket_test_simple['CustomerDMACode'] = bracket_test_simple['CustomerDMACode'].astype(str).str[:-2]


,CustomerDMACode,RegionWinner_East,RegionWinner_West,SemifinalWinner_East_West
BracketEntryId,,,,
2074118,560,UConn,North Carolina,
2692634,820,UConn,North Carolina,
1252684,529,Iowa St.,Arizona,
1950205,819,UConn,North Carolina,
2756293,504,UConn,North Carolina,


In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

# Define training data (from bracket_training_simple)
X_train = bracket_training_simple[['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_West']]
y_train = bracket_training_simple['SemifinalWinner_East_West']

# Define test data (from bracket_test_simple)
X_test = bracket_test_simple[['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_West']]

# Define categorical features
cat_features = ['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_West']

# Convert categorical features to indices
for feature in cat_features:
    X_train[feature] = X_train[feature].astype('category').cat.codes
    X_test[feature] = X_test[feature].astype('category').cat.codes

# Encode target variable
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Initialize the model
model = xgb.XGBClassifier(
    objective='multi:softmax',
    n_jobs=-1,
    use_label_encoder=False,
    eval_metric='mlogloss',
    num_class=len(le.classes_)  # Specify number of classes
)

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'gamma': [0, 0.1, 0.2]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2)

# Perform grid search
grid_search.fit(X_train, y_train_encoded)

# Get the best model
best_model = grid_search.best_estimator_

# Print the best parameters
print("Best parameters found: ", grid_search.best_params_)

# After getting predictions, decode them back to team names
predictions = best_model.predict(X_test)
predictions_decoded = le.inverse_transform(predictions)
bracket_test_simple['SemifinalWinner_East_West'] = predictions_decoded

/tmp/ipykernel_12998/1662697256.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[feature] = X_train[feature].astype('category').cat.codes
/tmp/ipykernel_12998/1662697256.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[feature] = X_test[feature].astype('category').cat.codes
/tmp/ipykernel_12998/1662697256.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

Fitting 3 folds for each of 81 candidates, totalling 243 fits


/home/anand173/.conda/envs/cent7/2024.02-py311/llama2/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [11:44:25] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [9]:
bracket_test_simple['SemifinalWinner_East_West'] = best_model.predict(X_test).flatten()
bracket_test_simple

/tmp/ipykernel_40368/598591828.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bracket_test_simple['SemifinalWinner_East_West'] = best_model.predict(X_test).flatten()


,CustomerDMACode,RegionWinner_East,RegionWinner_West,SemifinalWinner_East_West
BracketEntryId,,,,
2074118,560,UConn,North Carolina,North Carolina
2692634,820,UConn,North Carolina,UConn
1252684,529,Iowa St.,Arizona,Iowa St.
1950205,819,UConn,North Carolina,UConn
2756293,504,UConn,North Carolina,UConn
...,...,...,...,...
2085292,789,UConn,Arizona,Arizona
615949,516,UConn,North Carolina,UConn
13727,862,BYU,Arizona,Arizona


## SemiFinal 2 Predictions

In [10]:
bracket_training_semifinal2 = bracket_training[['CustomerDMACode', 'RegionWinner_South', 'RegionWinner_Midwest', 'SemifinalWinner_South_Midwest', 'BracketEntryId']]
bracket_training_semifinal2['CustomerDMACode'] = bracket_training_semifinal2['CustomerDMACode'].astype(str).str[:-2]
bracket_training_semifinal2.set_index('BracketEntryId', inplace=True)
bracket_training_semifinal2.head()

/tmp/ipykernel_40368/638671133.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bracket_training_semifinal2['CustomerDMACode'] = bracket_training_semifinal2['CustomerDMACode'].astype(str).str[:-2]


,CustomerDMACode,RegionWinner_South,RegionWinner_Midwest,SemifinalWinner_South_Midwest
BracketEntryId,,,,
1723503,698,James Madison,Tennessee,Tennessee
963479,563,Kentucky,Kansas,Kentucky
810038,511,Houston,Purdue,Purdue
3384825,641,NC State,Purdue,NC State
2828017,514,Marquette,Creighton,Marquette


In [11]:
bracket_test_semifinal2 = bracket_test[['CustomerDMACode', 'RegionWinner_South', 'RegionWinner_Midwest', 'SemifinalWinner_South_Midwest', 'BracketEntryId']]
bracket_test_semifinal2['CustomerDMACode'] = bracket_test_semifinal2['CustomerDMACode'].astype(str).str[:-2]
bracket_test_semifinal2.set_index('BracketEntryId', inplace=True)
bracket_test_semifinal2.head()

/tmp/ipykernel_40368/3465230186.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bracket_test_semifinal2['CustomerDMACode'] = bracket_test_semifinal2['CustomerDMACode'].astype(str).str[:-2]


,CustomerDMACode,RegionWinner_South,RegionWinner_Midwest,SemifinalWinner_South_Midwest
BracketEntryId,,,,
2074118,560,Houston,Purdue,
2692634,820,Duke,Kansas,
1252684,529,Kentucky,Creighton,
1950205,819,Houston,Purdue,
2756293,504,Marquette,Creighton,


In [12]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV

# Define training data (from bracket_training_semifinal2)
X_train = bracket_training_semifinal2[['CustomerDMACode', 'RegionWinner_South', 'RegionWinner_Midwest']]
y_train = bracket_training_semifinal2['SemifinalWinner_South_Midwest']

# Define test data (from bracket_test_semifinal2)
X_test = bracket_test_semifinal2[['CustomerDMACode', 'RegionWinner_South', 'RegionWinner_Midwest']]

# Define categorical features
cat_features = ['CustomerDMACode', 'RegionWinner_South', 'RegionWinner_Midwest']

# Initialize the model
model = CatBoostClassifier(
    cat_features=cat_features,
    task_type="GPU",  # Enables CUDA
    verbose=0
)

# Define the parameter grid
param_grid = {
    'iterations': [300],
    'depth': [3],
    'learning_rate': [0.1]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2)

# Perform grid search
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Print the best parameters
print("Best parameters found: ", grid_search.best_params_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END .........depth=3, iterations=300, learning_rate=0.1; total time=  26.0s
[CV] END .........depth=3, iterations=300, learning_rate=0.1; total time=  25.3s
[CV] END .........depth=3, iterations=300, learning_rate=0.1; total time=  26.7s
Best parameters found:  {'depth': 3, 'iterations': 300, 'learning_rate': 0.1}


In [13]:
bracket_test_semifinal2['SemifinalWinner_South_Midwest'] = best_model.predict(X_test).flatten()
bracket_test_semifinal2

/tmp/ipykernel_40368/4193702743.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bracket_test_semifinal2['SemifinalWinner_South_Midwest'] = best_model.predict(X_test).flatten()


,CustomerDMACode,RegionWinner_South,RegionWinner_Midwest,SemifinalWinner_South_Midwest
BracketEntryId,,,,
2074118,560,Houston,Purdue,Houston
2692634,820,Duke,Kansas,Duke
1252684,529,Kentucky,Creighton,Kentucky
1950205,819,Houston,Purdue,Houston
2756293,504,Marquette,Creighton,Marquette
...,...,...,...,...
2085292,789,Houston,Purdue,Houston
615949,516,Kentucky,Purdue,Purdue
13727,862,Houston,Creighton,Houston


In [14]:
finals_training_df = bracket_training[['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 
                                       'RegionWinner_Midwest', 'BracketEntryId', 'SemifinalWinner_East_West', 'SemifinalWinner_South_Midwest', 
                                       'NationalChampion']]

finals_training_df['CustomerDMACode'] = finals_training_df['CustomerDMACode'].astype(str).str[:-2]

finals_training_df

/tmp/ipykernel_40368/493096718.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finals_training_df['CustomerDMACode'] = finals_training_df['CustomerDMACode'].astype(str).str[:-2]


,CustomerDMACode,RegionWinner_East,RegionWinner_West,RegionWinner_South,RegionWinner_Midwest,BracketEntryId,SemifinalWinner_East_West,SemifinalWinner_South_Midwest,NationalChampion
0,698,UConn,Arizona,James Madison,Tennessee,1723503,UConn,Tennessee,Tennessee
1,563,UConn,Baylor,Kentucky,Kansas,963479,UConn,Kentucky,UConn
2,511,UConn,Baylor,Houston,Purdue,810038,UConn,Purdue,Purdue
3,641,Iowa St.,Saint Mary's,NC State,Purdue,3384825,Iowa St.,NC State,NC State
4,514,Auburn,North Carolina,Marquette,Creighton,2828017,North Carolina,Marquette,North Carolina
...,...,...,...,...,...,...,...,...,...
129997,751,Iowa St.,North Carolina,Kentucky,Tennessee,3602437,Iowa St.,Tennessee,Iowa St.
129998,618,Iowa St.,Alabama,Houston,Tennessee,2260739,Alabama,Houston,Houston
129999,811,UConn,North Carolina,Houston,Creighton,1298238,UConn,Houston,UConn
130000,630,UConn,Alabama,Kentucky,Purdue,681594,UConn,Purdue,Purdue


In [15]:
finals_test_df = bracket_test[['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 
                                       'RegionWinner_Midwest', 'BracketEntryId']]
finals_test_df.set_index('BracketEntryId', inplace=True)

# Select only required columns
finals_test_df = finals_test_df[['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 'RegionWinner_Midwest']]

# Merge only the required columns from semifinal DataFrames
finals_test_df = finals_test_df.join(bracket_test_simple[['SemifinalWinner_East_West']], how='left')
finals_test_df = finals_test_df.join(bracket_test_semifinal2[['SemifinalWinner_South_Midwest']], how='left')
finals_test_df['CustomerDMACode'] = finals_test_df['CustomerDMACode'].astype(str).str[:-2]

finals_test_df["NationalChampion"] = ""
finals_test_df


,CustomerDMACode,RegionWinner_East,RegionWinner_West,RegionWinner_South,RegionWinner_Midwest,SemifinalWinner_East_West,SemifinalWinner_South_Midwest,NationalChampion
BracketEntryId,,,,,,,,
2074118,560,UConn,North Carolina,Houston,Purdue,North Carolina,Houston,
2692634,820,UConn,North Carolina,Duke,Kansas,UConn,Duke,
1252684,529,Iowa St.,Arizona,Kentucky,Creighton,Iowa St.,Kentucky,
1950205,819,UConn,North Carolina,Houston,Purdue,UConn,Houston,
2756293,504,UConn,North Carolina,Marquette,Creighton,UConn,Marquette,
...,...,...,...,...,...,...,...,...
2085292,789,UConn,Arizona,Houston,Purdue,Arizona,Houston,
615949,516,UConn,North Carolina,Kentucky,Purdue,UConn,Purdue,
13727,862,BYU,Arizona,Houston,Creighton,Arizona,Houston,


In [16]:
X_train = finals_training_df[['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 
                              'RegionWinner_Midwest', 'SemifinalWinner_East_West', 'SemifinalWinner_South_Midwest']]
y_train = finals_training_df['NationalChampion']

# Define test data (from finals_test_df)
X_test = finals_test_df[['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 
                         'RegionWinner_Midwest', 'SemifinalWinner_East_West', 'SemifinalWinner_South_Midwest']]

# Define categorical features
cat_features = ['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 
                'RegionWinner_Midwest', 'SemifinalWinner_East_West', 'SemifinalWinner_South_Midwest']

# Define parameter grid for grid search
param_grid = {
    'iterations': [200, 300, 400],
    'depth': [3, 4,5],
    'learning_rate': [0.1],
    'cat_features': [cat_features],
    'task_type': ['GPU']
}

# Initialize the model
model = CatBoostClassifier(
    cat_features=cat_features,
    task_type="GPU",  # Enables CUDA
    verbose=0
)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2)

# Perform grid search
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Print the best parameters
print("Best parameters found: ", grid_search.best_params_)

# Make predictions with the best model
finals_test_df['NationalChampion'] = best_model.predict(X_test).flatten()

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] END cat_features=['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 'RegionWinner_Midwest', 'SemifinalWinner_East_West', 'SemifinalWinner_South_Midwest'], depth=3, iterations=200, learning_rate=0.1, task_type=GPU; total time=  25.5s
[CV] END cat_features=['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 'RegionWinner_Midwest', 'SemifinalWinner_East_West', 'SemifinalWinner_South_Midwest'], depth=3, iterations=200, learning_rate=0.1, task_type=GPU; total time=  25.0s
[CV] END cat_features=['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 'RegionWinner_Midwest', 'SemifinalWinner_East_West', 'SemifinalWinner_South_Midwest'], depth=3, iterations=200, learning_rate=0.1, task_type=GPU; total time=  26.3s
[CV] END cat_features=['CustomerDMACode', 'RegionWinner_East', 'RegionWinner_West', 'RegionWinner_South', 'RegionWinner_Midwest

In [17]:
# Select only the required columns, keeping BracketEntryId as the index
final_selection = finals_test_df[['SemifinalWinner_East_West', 'SemifinalWinner_South_Midwest', 'NationalChampion']]
final_selection

,SemifinalWinner_East_West,SemifinalWinner_South_Midwest,NationalChampion
BracketEntryId,,,
2074118,North Carolina,Houston,North Carolina
2692634,UConn,Duke,UConn
1252684,Iowa St.,Kentucky,Iowa St.
1950205,UConn,Houston,UConn
2756293,UConn,Marquette,UConn
...,...,...,...
2085292,Arizona,Houston,Arizona
615949,UConn,Purdue,UConn
13727,Arizona,Houston,Houston


In [18]:
final_selection.to_csv('catboostmodel.csv')